In [2]:
from shapely.geometry import CAP_STYLE
from qiskit_metal import draw, Dict  # , QComponent
from qiskit_metal.qlibrary.core import QComponent 
import math 

class Whisper_gallery_resonator( QComponent ):
    """ A double chip resonator devic "vrunda_design .ipynb"e "
    default_options = ( radius = pass,
                        outer_cir_pos_x = ,
                        outer_cir_pos_y = ,
                        transmon_margin = ,
                        trans_rect_width = ,
                        trans_rect_len = ,
                        trans_rect_pos_x = ,
                        trans_rect_pos_y = ,
                        inner_cir_pos_x = ,
                        inner_cir_pos_y = ,
                        layer = ,
                        chip = ,
                    )"""
    component_metadata = Dict(short_name='WGR',
                              _qgeometry_table_path='True',
                              _qgeometry_table_poly='True',
                              _qgeometry_table_junction='True')
    """Component metadata"""
    default_options = dict( radius = '7.5mm',
                            outer_cir_pos_x = '0mm',
                            outer_cir_pos_y = '0mm',
                            inner_cir_rotation = '45',
                            inner_cir_radius = '5mm',
                            transmon_margin = '0.05mm',
                            trans_orientation = '90',
                            trans_qubit_rotation = '0',
                            trans_rect_height = '0.23mm',
                            trans_rect_width = '0.5mm',
                            trans_rect_pos_x = '7.35mm',
                            trans_rect_pos_y = '0mm',
                            trans_pad_width = '0.45mm',
                            trans_pad_height = '0.12mm',
                            layer = '1',
                            chip = 'main')
    def make( self):
        p = self.p
        ################################## defining outer_circle #################################################################
        outer_circle = draw.shapely.geometry.Point(p.outer_cir_pos_x, p.outer_cir_pos_y).buffer( p.radius)

        #################################### defining transmon ###################################################################                                                                                     
        r_transmon = p.outer_cir_pos_x + p.radius - p.transmon_margin - p.trans_rect_width/2
        pos_x = (r_transmon) * math.cos( p.trans_orientation * math.pi / 180 ) 
        pos_y = (r_transmon) * math.sin( p.trans_orientation * math.pi / 180 ) 
        r1 = draw.rectangle( p.trans_rect_width, p.trans_rect_height,pos_x, pos_y)
        r2 = draw.rectangle( p.trans_pad_width, p.trans_pad_height, pos_x, pos_y)
        transmon = draw.subtract( r1, r2)
        # trans_rotation= draw.rotate( trans, p.trans_qubit_rotation, origin = ( pos_x, pos_y))
        ################################### josephson_junction ###################################################################
        inductor_width = p.trans_rect_width - p.trans_pad_width
        ################################## condition for inner circle #############################################################
        rt = p.outer_cir_pos_x + p.radius - (2*p.transmon_margin)-p.trans_rect_width
        inner_cir_pos_x = (rt-p.inner_cir_radius) * math.cos( p.inner_cir_rotation * math.pi / 180 ) 
        inner_cir_pos_y = (rt-p.inner_cir_radius ) * math.sin( p.inner_cir_rotation * math.pi / 180 ) 
        if( math.sqrt(inner_cir_pos_x**2 + inner_cir_pos_y**2) + p.inner_cir_radius > rt):
            return False
        ''' **************************** Nearest semi-intermediate circle coordinates *************************************************
        # x_nearest = rt* math.cos( p.inner_cir_rotation * math.pi / 180 )
        # y_nearest = rt* math.sin( p.inner_cir_rotation * math.pi / 180 )
        # ri = math.sqrt( p.inner_cir_pos_x**2 + p.inner_cir_pos_y**2)
        # if ri>=rt:
        #     return False
        # inner_cir_radius = rt-ri
        ## defining inner circle 
        # inner_radius = p.radius -  (2* p.transmon_margin + p.trans_rect_len)-( p.outer_cir_pos_x - p.inner_cir_pos_x)'''
        ################################### inner_circle ##########################################################################
        inner_circle = draw.shapely.geometry.Point( inner_cir_pos_x, inner_cir_pos_y).buffer( p.inner_cir_radius )  

        #################################### phi ##################################################################################
        diff_rect_pad_width = (p.trans_rect_width - p.trans_pad_width)/2
        R1 = r_transmon - p.trans_pad_width/2  - diff_rect_pad_width/2
        a = diff_rect_pad_width/2 + p.trans_pad_width/2
        # phi = math.acos( (R1**2 + r_transmon**2 - a**2) / 2 *r_transmon*R1) + (p.trans_orientation * 180)/math.pi
        
        #################################### josephson_junction ######################################################################
        
        r_1 = R1 - diff_rect_pad_width/2
        r_2 = R1 + diff_rect_pad_width/2
        # R1_x = R1 * math.cos( p.trans_orientation * math.pi / 180 ) 
        # R1_y = R1 * math.sin( p.trans_orientation * math.pi /180 )
        r_1_x = r_1 * math.cos(p.trans_orientation * math.pi / 180 ) 
        r_1_y = r_1 * math.sin(p.trans_orientation  * math.pi / 180 ) 
        r_2_x = r_2 * math.cos(p.trans_orientation  * math.pi / 180 ) 
        r_2_y = r_2 * math.sin(p.trans_orientation  * math.pi / 180 ) 

        joseph = draw.LineString([(r_2_x, r_2_y),(r_1_x, r_1_y)])

        
        final_design_1 = draw.subtract( outer_circle, r1)
        final_design_2 = draw.union(final_design_1 , r2)
        final_design_3 = draw.subtract( final_design_2, inner_circle)
        objects = [final_design_3,joseph]
        objects = draw.rotate(objects, p.orientation, origin=(p.outer_cir_pos_x, p.outer_cir_pos_y))
        objects = draw.translate(objects, p.pos_x, p.pos_y)
        [final_design_3,joseph] = objects
        self.add_qgeometry('poly', {'final_design_3': final_design_3},
                           subtract=True,
                           helper=p.helper,
                           layer=p.layer,
                           chip=p.chip)
        self.add_qgeometry('junction', {'joseph': joseph},
                           subtract=True,
                           width= diff_rect_pad_width,
                           layer=p.layer,
                           chip=p.chip)
        


In [3]:
from qiskit_metal import designs, MetalGUI
design  = designs.DesignFlipChip()
gui = MetalGUI( design ) 
gui.overwrite_enabled = True




In [4]:
design.overwrite_enabled = True
q1 =  Whisper_gallery_resonator( design, "Q1", options = Dict(inner_cir_rotation = '0', trans_orientation = '0',chip = 'Q_chip') )
gui.rebuild()
gui.autoscale()

01:18PM 11s ERROR [__getitem__]: 
 Component name = `Q1`
 Option name    = `helper`
01:18PM 11s ERROR [__getitem__]: 
 Component name = `Q1`
 Option name    = `helper`


In [5]:
import pyEPR as epr

In [6]:
from qiskit_metal.analyses.quantization import EPRanalysis

In [ ]:
from shapely.geometry import CAP_STYLE
from qiskit_metal import draw, Dict  # , QComponent
from qiskit_metal.qlibrary.core import QComponent 
import math 

class Whisper_gallery_resonator_u( QComponent ):
    """ A double chip resonator device "
    default_options = ( radius = pass,
                        outer_cir_pos_x = ,
                        outer_cir_pos_y = ,
                        transmon_margin = ,
                        trans_rect_width = ,
                        trans_rect_len = ,
                        trans_rect_pos_x = ,
                        trans_rect_pos_y = ,
                        inner_cir_pos_x = ,
                        inner_cir_pos_y = ,
                        layer = ,
                        chip = ,
                    )"""
    component_metadata = Dict(short_name='WGR',
                              _qgeometry_table_path='True',
                              _qgeometry_table_poly='True',
                              _qgeometry_table_junction='True')
    """Component metadata"""
    default_options = dict( radius = '7.5mm',
                            outer_cir_pos_x = '0mm',
                            outer_cir_pos_y = '0mm',
                            inner_cir_rotation = '45',
                            inner_cir_radius = '5mm',
                            transmon_margin = '0.05mm',
                            # trans_orientation = '90',
                            # trans_qubit_rotation = '0',
                            trans_rect_len = '0.5mm',
                            # trans_rect_width = '0.23mm',
                            # trans_rect_pos_x = '7.35mm',
                            # trans_rect_pos_y = '0mm',
                            # trans_pad_width = '0.12mm',
                            # trans_pad_len = '0.45mm',
                            layer = '1',
                            chip = 'main')
    def make( self):
        p = self.p
        ################################## defining outer_circle #################################################################
        outer_circle = draw.shapely.geometry.Point(p.outer_cir_pos_x, p.outer_cir_pos_y).buffer( p.radius)

        #################################### defining transmon ###################################################################                                                                                     
        # r_transmon = p.outer_cir_pos_x + p.radius - p.transmon_margin - p.trans_rect_len/2
        # pos_x = (r_transmon) * math.cos( p.trans_orientation * math.pi / 180 ) 
        # pos_y = (r_transmon) * math.sin( p.trans_orientation * math.pi / 180 ) 
        # r1 = draw.rectangle( p.trans_rect_width, p.trans_rect_len,pos_x, pos_y)
        # r2 = draw.rectangle( p.trans_pad_width, p.trans_pad_len, pos_x, pos_y)
        # transmon = draw.subtract( r1, r2)
        # trans_rotation= draw.rotate( trans, p.trans_qubit_rotation, origin = ( pos_x, pos_y))
        ################################### josephson_junction ###################################################################
        # inductor_width = p.trans_rect_width - p.trans_pad_width
        ################################## condition for inner circle #############################################################
        rt = p.outer_cir_pos_x + p.radius - (2*p.transmon_margin)-p.trans_rect_len
        inner_cir_pos_x = (rt-p.inner_cir_radius) * math.cos( p.inner_cir_rotation * math.pi / 180 ) 
        inner_cir_pos_y = (rt-p.inner_cir_radius ) * math.sin( p.inner_cir_rotation * math.pi / 180 ) 
        if( math.sqrt(inner_cir_pos_x**2 + inner_cir_pos_y**2) + p.inner_cir_radius > rt):
            return False
        ''' **************************** Nearest semi-intermediate circle coordinates *************************************************
        # x_nearest = rt* math.cos( p.inner_cir_rotation * math.pi / 180 )
        # y_nearest = rt* math.sin( p.inner_cir_rotation * math.pi / 180 )
        # ri = math.sqrt( p.inner_cir_pos_x**2 + p.inner_cir_pos_y**2)
        # if ri>=rt:
        #     return False
        # inner_cir_radius = rt-ri
        ## defining inner circle 
        # inner_radius = p.radius -  (2* p.transmon_margin + p.trans_rect_len)-( p.outer_cir_pos_x - p.inner_cir_pos_x)'''
        ################################### inner_circle ##########################################################################
        inner_circle = draw.shapely.geometry.Point( inner_cir_pos_x, inner_cir_pos_y).buffer( p.inner_cir_radius )  
        # final_design_1 = draw.subtract( outer_circle, r1)
        # final_design_2 = draw.union(final_design_1 , r2)
        final_design_3 = draw.subtract( outer_circle, inner_circle)
        objects = [final_design_3]
        objects = draw.rotate(objects, p.orientation, origin=(p.outer_cir_pos_x, p.outer_cir_pos_y))
        objects = draw.translate(objects, p.pos_x, p.pos_y)
        [final_design_3] = objects
        self.add_qgeometry('poly', {'final_design_3': final_design_3},
                           subtract=True,
                           helper=p.helper,
                           layer=p.layer,
                           chip=p.chip)


In [ ]:
gui.overwrite_enabled = True 
q2  =  Whisper_gallery_resonator_u( design, 'Q2', options = Dict( inner_cir_rotation = '0', trans_orientation = '0',chip = 'C_chip'))
gui.rebuild()
gui.autoscale()

In [ ]:
pinfo = epr.ProjectInfo( project_path = r'C:\Users\uday mathur\Desktop\qiskit metal jupyter\qiskit_metal_practice_31',
                    project_name = 'flip_chip_error_recog_1',
                    design_name = 'Flip_Chip')

01:18PM 39s ERROR [__getitem__]: 
 Component name = `Q2`
 Option name    = `helper`
01:18PM 39s ERROR [__getitem__]: 
 Component name = `Q1`
 Option name    = `helper`
01:18PM 39s ERROR [__getitem__]: 
 Component name = `Q2`
 Option name    = `helper`


In [12]:
eig_res = EPRanalysis( design, 'hfss')
hfss = eig_res.sim.renderer


In [13]:
hfss.start()

INFO 01:19PM [connect_project]: Connecting to Ansys Desktop API...
INFO 01:19PM [load_ansys_project]: 	Opened Ansys App
INFO 01:19PM [load_ansys_project]: 	Opened Ansys Desktop v2022.2.0
INFO 01:19PM [load_ansys_project]: 	Opened Ansys Project
	Folder:    C:/Users/uday mathur/Documents/Ansoft/
	Project:   Project25
INFO 01:19PM [connect_design]: No active design found (or error getting active design).
INFO 01:19PM [connect]: 	 Connected to project "Project25". No design detected


True

In [14]:
hfss.activate_ansys_design( "flip_chip", 'eigenmode')
hfss.render_design()

01:19PM 38s WARNING [activate_ansys_design]: The design_name=flip_chip was not in active project.  Designs in active project are: 
[].  A new design will be added to the project.  
INFO 01:19PM [connect_design]: 	Opened active design
	Design:    flip_chip [Solution type: Eigenmode]
WARNING 01:19PM [connect_setup]: 	No design setup detected.
WARNING 01:19PM [connect_setup]: 	Creating eigenmode default setup.
INFO 01:19PM [get_setup]: 	Opened setup `Setup`  (<class 'pyEPR.ansys.HfssEMSetup'>)


INFO 01:22PM [connect_project]: Connecting to Ansys Desktop API...
INFO 01:22PM [load_ansys_project]: 	File path to HFSS project found.
INFO 01:22PM [load_ansys_project]: 	Opened Ansys App
INFO 01:22PM [load_ansys_project]: 	Opened Ansys Desktop v2022.2.0
INFO 01:22PM [load_ansys_project]: 	Opened Ansys Project
	Folder:    C:/Users/uday mathur/Desktop/qiskit metal jupyter/qiskit_metal_practice_31/
	Project:   flip_chip_error_recog_1
INFO 01:22PM [connect_design]: 	Opened active design
	Design:    Flip_Chip [Solution type: Eigenmode]
INFO 01:22PM [get_setup]: 	Opened setup `Setup`  (<class 'pyEPR.ansys.HfssEMSetup'>)
INFO 01:22PM [connect]: 	Connected to project "flip_chip_error_recog_1" and design "flip_chip" 😀 




design.chips['Q_chip']['size']['center_z'] = '10 um'

In [17]:
design.chips['Q_chip']['size']['center_z'] = '10 um'

In [18]:
eig_res.sim.setup.max_passes = 20
eig_res.sim.setup.vars.Lj = '10nH'

In [19]:
eig_res.sim.setup.n_modes = 3

In [20]:
eig_res.setup.dissipatives.dielectrics_bulk = ['Q_chip', 'C_chip']

In [21]:
eig_res.sim.run()

INFO 01:22PM [get_setup]: 	Opened setup `Setup`  (<class 'pyEPR.ansys.HfssEMSetup'>)
INFO 01:22PM [analyze]: Analyzing setup Setup
01:23PM 12s INFO [get_f_convergence]: Saved convergences to C:\Users\uday mathur\Desktop\qiskit metal jupyter\qiskit_metal_practice_31\hfss_eig_f_convergence.csv


In [22]:
eig_res.sim.print_run_args()

This analysis object run with the following kwargs:
{}



In [23]:
eig_res.sim.convergence_f

,re(Mode(1)) [g],re(Mode(2)) [g],re(Mode(3)) [g]
Pass [],,,
1,5.690569,5.774563,6.967915
2,8.857041,9.177725,12.247266
3,9.101726,9.556209,12.249715
4,9.153372,9.636102,12.250462
5,9.185892,9.664056,12.250695
6,NaN,NaN,NaN
7,NaN,NaN,NaN
8,NaN,NaN,NaN
9,NaN,NaN,NaN


In [24]:
eig_res.setup

{'junctions': {'jj': {'Lj_variable': 'Lj',
   'Cj_variable': 'Cj',
   'rect': '',
   'line': ''}},
 'dissipatives': {'dielectrics_bulk': ['Q_chip', 'C_chip']},
 'cos_trunc': 8,
 'fock_trunc': 7,
 'sweep_variable': 'Lj'}

In [25]:
eig_res.setup.junctions.jj.rect = 'JJ_rect_Lj_Q1_rect_jj'

In [26]:
eig_res.setup.junctions.jj.line = 'JJ_Lj_Q1_rect_jj_'

In [27]:
eig_res.setup

{'junctions': {'jj': {'Lj_variable': 'Lj',
   'Cj_variable': 'Cj',
   'rect': 'JJ_rect_Lj_Q1_rect_jj',
   'line': 'JJ_Lj_Q1_rect_jj_'}},
 'dissipatives': {'dielectrics_bulk': ['Q_chip', 'C_chip']},
 'cos_trunc': 8,
 'fock_trunc': 7,
 'sweep_variable': 'Lj'}

In [36]:
pinfo.design
pinfo.project
pinfo.connect_project
pinfo.connect
pinfo.check_connected()
pinfo.connect()

INFO 01:27PM [connect_project]: Connecting to Ansys Desktop API...
INFO 01:27PM [load_ansys_project]: 	File path to HFSS project found.
INFO 01:27PM [load_ansys_project]: 	Opened Ansys App
INFO 01:27PM [load_ansys_project]: 	Opened Ansys Desktop v2022.2.0
INFO 01:27PM [load_ansys_project]: 	Opened Ansys Project
	Folder:    C:/Users/uday mathur/Desktop/qiskit metal jupyter/qiskit_metal_practice_31/
	Project:   flip_chip_error_recog_1
INFO 01:27PM [connect_design]: 	Opened active design
	Design:    flip_chip [Solution type: Eigenmode]
INFO 01:27PM [get_setup]: 	Opened setup `Setup`  (<class 'pyEPR.ansys.HfssEMSetup'>)
INFO 01:27PM [connect]: 	Connected to project "flip_chip_error_recog_1" and design "flip_chip" 😀 



In [38]:
pinfo.get_dm()

(<pyEPR.ansys.HfssDesign at 0x20cdc6f2dd0>,
 <pyEPR.ansys.HfssModeler at 0x20cdc37c190>)

In [64]:
pinfo.get_all_variables_names()
pinfo.get_all_object_names()

['C_chip',
 'Q_chip',
 'sample_holder',
 'JJ_rect_Lj_Q1_joseph',
 'ground_C_chip_plane',
 'ground_Q_chip_plane',
 'JJ_Lj_Q1_joseph_']

In [65]:
# pinfo = hfss.pinfo
pinfo.junctions['jj'] = {'Lj_variable': 'Lj', 'rect': 'JJ_rect_Lj_Q1_joseph',
                             'line': 'JJ_Lj_Q1_joseph_',  'Cj_variable': 'Cj'}


In [66]:
pinfo.junctions.items()

dict_items([('jj', {'Lj_variable': 'Lj', 'rect': 'JJ_rect_Lj_Q1_joseph', 'line': 'JJ_Lj_Q1_joseph_', 'Cj_variable': 'Cj'})])

In [68]:
pinfo.validate_junction_info()

In [69]:
eprh = epr.DistributedAnalysis( pinfo)

Design "flip_chip" info:
	# eigenmodes    3
	# variations    1


In [73]:
ℰ_total  = eprh.calc_energy_electric(obj='AllObjects')
ℰ_total

2.09341447266132e-22

In [76]:
ℰ_substr = eprh.calc_energy_electric(obj='substrate')
ℰ_substr

com_error: (-2147352567, 'Exception occurred.', (0, None, None, None, 0, -2147024365), None)

In [77]:
eprh.do_EPR_analysis( append_analysis = True )

  options=pd.Series(get_instance_vars(self.options)),




Variation 0  [1/1]


  Ljs = pd.Series({})

  Cjs = pd.Series({})

  _Om = pd.Series({})




  Mode 0 at 9.19 GHz   [1/3]
    Calculating ℰ_magnetic,

  Sj = pd.Series({})

  Qp = pd.Series({})



...

  Qp = pd.Series({})

  _Om = pd.Series({})



		Energy fraction (Lj over Lj&Cj)= 93.13%
	jj              1.11451e-13  (+)        8.21848e-15
		(U_tot_cap-U_tot_ind)/mean=-0.00%

  Mode 2 at 12.25 GHz   [3/3]
    Calculating ℰ_magnetic,

  Sj = pd.Series({})

  Qp = pd.Series({})



ℰ_electric
       (ℰ_E-ℰ_H)/ℰ_E       ℰ_E       ℰ_H
               -0.1%   8.85e-20 8.855e-20

    Calculating junction energy participation ration (EPR)
	method=`line_voltage`. First estimates:
	junction        EPR p_2j   sign s_2j    (p_capacitive)
		Energy fraction (Lj over Lj&Cj)= 89.41%
	jj              2.53754e-12  (+)        3.00694e-13
		(U_tot_cap-U_tot_ind)/mean=-0.03%


com_error: (-2147352567, 'Exception occurred.', (0, None, None, None, 0, -2147024382), None)

In [80]:
eprh.pinfo

In [84]:
eprh.fields

In [85]:
eprh.setup.get_fields()

In [141]:
eprh.results

{'0': {'Pm':              jj
  0  3.343007e-09
  1  1.114508e-13
  2  2.537541e-12,
  'Pm_cap':              jj
  0  2.227254e-10
  1  8.218484e-15
  2  3.006936e-13,
  'Sm':    s_jj
  0    -1
  1     1
  2     1,
  'Om':                  0         1          2
  freq_GHz  9.185892  9.664056  12.250695,
  'sols':             U_H           U_E
  0  2.093414e-22  2.093414e-22
  1  3.333037e-20  3.333036e-20
  2  1.770959e-19  1.769942e-19,
  'Qm_coupling': Empty DataFrame
  Columns: []
  Index: [0, 1, 2],
  'Ljs': jj    1.000000e-08
  dtype: float64,
  'Cjs': jj    2.000000e-15
  dtype: float64,
  'Qs': 0    inf
  1    inf
  2    inf
  dtype: float64,
  'freqs_hfss_GHz': 0     9.185892
  1     9.664056
  2    12.250695
  dtype: float64,
  'hfss_variables': _Cj     0fF
  _Lj    10nH
  dtype: object,
  'modes': range(0, 3),
  'I_peak': 0    0   -8.365584e-12
  dtype: float64
  1    0    6.094829e-13
  dtype: float64
  2    0    6.701715e-12
  dtype: float64
  dtype: object,
  'V_peak': 0  

In [92]:
eprh.n_modes, eprh.modes 

(3, range(0, 3))

In [94]:
eprh.variations

['0']

In [96]:
eprh.variations_analyzed

[]

In [97]:
eprh._nominal_variation

"Cj='0fF' Lj='10nH'"

In [98]:
eprh._list_variations

("Cj='0fF' Lj='10nH'",)

In [99]:
eprh._hfss_variables

{'0': _Cj     0fF
 _Lj    10nH
 dtype: object}

In [100]:
eprh._previously_analyzed

set()

In [101]:
eprh.update_ansys_info()

In [102]:
eprh.design.name

'flip_chip'

In [104]:
eprh.project.name

'flip_chip_error_recog_1'

In [107]:
eprh.data_dir

WindowsPath('C:/data-pyEPR/flip_chip_error_recog_1/flip_chip')

In [112]:
eprh.file_name

In [113]:
eprh.setup_data()

In [119]:
eprh.setup

In [121]:
eprh.pinfo.design.name

'flip_chip'

In [122]:
eprh.pinfo.junctions

{'jj': {'Lj_variable': 'Lj',
  'rect': 'JJ_rect_Lj_Q1_joseph',
  'line': 'JJ_Lj_Q1_joseph_',
  'Cj_variable': 'Cj'}}

In [125]:
eprh.pinfo.ports

{}

In [126]:
eprh.pinfo.app

In [127]:
eprh.pinfo.options

{'method_calc_P_mj': 'line_voltage', 'save_mesh_stats': True}

In [136]:
eprh.calc_p_junction_single('0','0'),eprh.calc_p_junction_single('1','0'),eprh.calc_p_junction_single('2','0')

    p_j_0 = -0.0005746769898997552
    p_j_1 = -0.0005746769898997552
    p_j_2 = -0.0005746769898997552


(OrderedDict([('pj_0', 0.0005746769898997552)]),
 OrderedDict([('pj_1', 0.0005746769898997552)]),
 OrderedDict([('pj_2', 0.0005746769898997552)]))

In [139]:
eprh.get_freqs_bare_pd( '0', frame = True)

,Freq. (GHz),Quality Factor
mode,,
0,9.185892,inf
1,9.664056,inf
2,12.250695,inf


In [145]:
eprh.solutions.eigenmodes('0')

([9.18589191755, 9.66405636233, 12.2506953646], None)

In [148]:
eprh.get_ansys_frequencies_all()

Freq. (GHz)  Quality Factor
variation mode                             
0         0        9.185892             inf
          1        9.664056             inf
          2       12.250695             inf

In [155]:
eprh._get_lv()

['Cj:=', '0fF', 'Lj:=', '10nH']

In [156]:
eprh.get_variations()

OrderedDict([('0', "Cj='0fF' Lj='10nH'")])

In [157]:
eprh.get_variation_string()

"Cj='0fF' Lj='10nH'"

In [158]:
eprh.get_ansys_variations()

("Cj='0fF' Lj='10nH'",)

In [159]:
eprh.get_ansys_variables()

variation,0
Cj,0fF
Lj,10nH


In [163]:
E = eprh.calc_energy_electric()

In [164]:
B = eprh. calc_energy_magnetic()

In [165]:
(E-B)/E

-0.0005746769898997552

In [169]:
eprh.calc_p_electric_volume(name_dielectric3D = 'flip_chip')

com_error: (-2147352567, 'Exception occurred.', (0, None, None, None, 0, -2147024365), None)